# Chinook Database SQL Q&A Agent

This notebook implements a natural language to SQL agent with:
- **Guardrails**: Validates questions before processing
- **SQL Generation**: Converts natural language to SQL queries
- **Natural Language Output**: Converts SQL results to human-readable answers
- **Safety**: Read-only queries with security checks

## 1. Database Schema Exploration

First, let's explore the Chinook database structure.

In [ ]:
import sqlite3

# Connect and list all tables
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = [t[0] for t in cursor.fetchall()]

print(f"Found {len(tables)} tables:")
for table in tables:
    print(f"  - {table}")

In [ ]:
# Display sample row from each table
for table in tables:
    cursor.execute(f"SELECT * FROM {table} LIMIT 1;")
    row = cursor.fetchone()
    print(f'\n-- Sample row from {table}:')
    if row:
        cols = [desc[0] for desc in cursor.description]
        print(dict(zip(cols, row)))
    else:
        print('-- (empty)')

conn.close()

## 2. Install Required Dependencies

In [ ]:
! pip install openai -q

## 3. SQL Agent with Guardrails and Natural Language Output

This agent includes:
- **Question validation**: Checks if the question is valid and safe
- **SQL generation**: Converts natural language to SQL
- **Safe execution**: Only allows SELECT queries
- **Natural language answer**: Converts results to readable text

In [ ]:
import sqlite3
import re
import os
from openai import OpenAI

# ======================
# CONFIG
# ======================
DB_PATH = "Chinook_Sqlite.sqlite"
# Get API key from environment variable
API_KEY = os.environ.get("OPENROUTER_API_KEY", "your-openrouter-api-key")

# AGENT MODELS
GROK_CODE = "x-ai/grok-code-fast-1"  # NOT FREE
SMALL_GEMMA = 'google/gemma-3n-e4b-it:free'
OPENAI_SMOL = "openai/gpt-oss-20b:free"
GEMINI_FLASH = "google/gemini-2.5-flash-lite-preview-09-2025"
QWEN_30BA3B = 'qwen/qwen3-30b-a3b:free'

SQL_GENERATOR_MODEL = GEMINI_FLASH  # Using free model by default
GUARDRAIL_MODEL = SMALL_GEMMA
ANSWER_MODEL = SMALL_GEMMA

# ======================
# LOAD SCHEMA
# ======================
with open(".\\schema.sql", "r", encoding="utf-16") as f:
    SCHEMA_DDL = f.read()

with open(".\\sample_rows.txt", "r", encoding="utf-8") as f:
    SAMPLE_ROWS = f.read()

# ======================
# AGENT PROMPTS
# ======================
GUARDRAIL_PROMPT = """You are a query validator. Determine if the user question:
- Is about data in the provided database schema
- Can be answered with a read-only SQL query
- Contains no malicious intent, PII, or out-of-scope requests

Answer ONLY "VALID" or "INVALID".

Schema:
{schema}

Question: {question}
"""

SQL_PROMPT = """You are a SQLite expert. Convert the question to a single SELECT query.

Rules:
- Use ONLY tables/columns from the schema
- NO markdown, NO explanations, NO comments
- Return ONLY the SQL query ending with ;

Schema:
{schema}

Question: {question}
"""

ANSWER_PROMPT = """You are a data analyst. Answer the user's original question using the SQL result.

Original question: {question}
SQL result (tab-separated):
{result}

Answer concisely in natural language. Do not mention SQL.
"""

# ======================
# HELPER FUNCTIONS
# ======================
def call_openrouter(model: str, prompt: str) -> str:
    """Call OpenRouter API with the given model and prompt."""
    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=API_KEY,
    )
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    return completion.choices[0].message.content.strip()

def validate_question(question: str) -> bool:
    """Validate if the question is safe and relevant."""
    prompt = GUARDRAIL_PROMPT.format(schema=SCHEMA_DDL, question=question)
    response = call_openrouter(GUARDRAIL_MODEL, prompt)
    return "VALID" in response.upper()

def generate_sql(question: str) -> str:
    """Generate SQL query from natural language question."""
    prompt = SQL_PROMPT.format(schema=SCHEMA_DDL, question=question)
    raw = call_openrouter(SQL_GENERATOR_MODEL, prompt)
    # Extract first SELECT...;
    match = re.search(r"(SELECT.*?;)", raw, re.DOTALL | re.IGNORECASE)
    return match.group(1) if match else raw.split("\n")[0]

def execute_sql_safe(sql: str):
    """Execute SQL with safety checks."""
    # 1. Enforce SELECT only
    if not re.match(r"^\s*SELECT\s", sql, re.IGNORECASE):
        raise ValueError("Only SELECT queries allowed")
    # 2. Block dangerous patterns
    dangerous = ["DROP", "DELETE", "INSERT", "UPDATE", "ATTACH", "PRAGMA", "--"]
    if any(kw in sql.upper() for kw in dangerous):
        raise ValueError("Query contains unsafe operations")
    # 3. Execute
    conn = sqlite3.connect(DB_PATH, uri=True)
    conn.row_factory = sqlite3.Row
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    conn.close()
    return rows

def format_results_as_tsv(rows):
    """Format SQL results as tab-separated values."""
    if not rows:
        return "(no results)"
    header = "\t".join(rows[0].keys())
    body = "\n".join("\t".join(str(v) for v in row) for row in rows)
    return f"{header}\n{body}"

def generate_natural_answer(question: str, tsv_result: str) -> str:
    """Generate natural language answer from SQL results."""
    prompt = ANSWER_PROMPT.format(question=question, result=tsv_result)
    return call_openrouter(ANSWER_MODEL, prompt)

print("Agent initialized successfully!")
print(f"Note: Make sure OPENROUTER_API_KEY environment variable is set.")

## 4. Ask Questions!

Now you can ask questions about the Chinook database in natural language.

In [ ]:
# ======================
# ASK YOUR QUESTION HERE
# ======================
USER_QUESTION = "List the top 5 artists by number of tracks"

print(f"❓ Question: {USER_QUESTION}\n")
print("=" * 80)

# --- STEP 1: GUARDRAILS ---
if not validate_question(USER_QUESTION):
    print("❌ Rejected: Question is invalid, off-topic, or unsafe.")
else:
    try:
        # --- STEP 2: GENERATE SQL ---
        sql_query = generate_sql(USER_QUESTION)
        print(f"🔍 Generated SQL:\n{sql_query}\n")
        print("=" * 80)

        # --- STEP 3: EXECUTE ---
        results = execute_sql_safe(sql_query)
        tsv_output = format_results_as_tsv(results)
        print(f"📊 Raw Results:\n{tsv_output}\n")
        print("=" * 80)

        # --- STEP 4: NATURAL LANGUAGE ANSWER ---
        natural_answer = generate_natural_answer(USER_QUESTION, tsv_output)
        print(f"✅ Answer:\n{natural_answer}")

    except Exception as e:
        print(f"💥 Error: {e}")

## 5. Example Questions to Try

```python
# Business questions
"Who are the top 5 customers by total spending?"
"What are the most popular genres by number of tracks?"
"List all employees and their managers"

# Analytical questions
"What is the average invoice total by country?"
"Who is the second least popular artist in the database?"
"How many tracks are there per genre?"
```

---

## 6. Streamlit Web App

The following cells create a Streamlit web application for the SQL agent.

In [ ]:
%%writefile app.py

import streamlit as st
import sqlite3
import re
import os
from openai import OpenAI

# ======================
# CONFIG
# ======================
DB_PATH = "Chinook_Sqlite.sqlite"
API_KEY = os.environ.get("OPENROUTER_API_KEY", "your-openrouter-api-key")

# Use real, available models
GUARD_MODEL = 'google/gemma-3n-e4b-it:free'
META_MODEL = "google/gemini-2.5-flash-lite-preview-09-2025"
SQL_MODEL = "google/gemini-2.5-flash-lite-preview-09-2025"
ANSWER_MODEL = 'google/gemma-3n-e4b-it:free'

# Load schema (MUST be UTF-8)
try:
    with open("schema.sql", "r", encoding="utf-16") as f:
        SCHEMA_DDL = f.read()
except Exception as e:
    st.error(f"❌ schema.sql missing or not UTF-16: {e}")
    st.stop()

# ======================
# PROMPTS
# ======================
GUARD_PROMPT = """You are a validator. Is the question:
- About the Chinook music store database?
- Safe and non-malicious?
Answer ONLY "VALID" or "INVALID".

Question: {question}
"""

META_PROMPT = """You are a database expert. Is this question asking about the database structure (e.g., tables, columns, relationships) rather than data?

Examples of YES: 
- "What tables are in the database?"
- "What columns does the Customer table have?"
- "Describe the schema."

Examples of NO:
- "How many customers are there?"
- "List artists from the 80s"

Answer ONLY "META" or "DATA".

Question: {question}
"""

SQL_PROMPT = """Generate a SQLite SELECT query. Rules:
- Use ONLY the provided schema
- NO explanations, markdown, or comments
- Output ONLY the SQL ending with ;

Schema:
{schema}

Question: {question}
"""

ANSWER_FROM_META_PROMPT = """Answer the user's question using ONLY the database schema below.

Question: {question}
Schema:
{schema}

Answer concisely in natural language.
"""

ANSWER_FROM_DATA_PROMPT = """Answer using the SQL result.

Question: {question}
Result (tab-separated):
{result}

Answer concisely. Do not mention SQL.
"""

# ======================
# HELPERS
# ======================
def call_openrouter(model: str, prompt: str) -> str:
    client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=API_KEY)
    resp = client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0,
        max_tokens=500,
    )
    return resp.choices[0].message.content.strip()

def safe_execute_sql(sql: str):
    if not re.match(r"^\s*SELECT\s", sql, re.IGNORECASE):
        raise ValueError("Only SELECT allowed")
    for kw in ["DROP", "DELETE", "INSERT", "UPDATE", "ATTACH", "PRAGMA"]:
        if kw in sql.upper():
            raise ValueError("Unsafe SQL")
    with sqlite3.connect(DB_PATH) as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        cur.execute(sql)
        return cur.fetchall()

def format_tsv(rows):
    if not rows:
        return "(no results)"
    header = "\t".join(rows[0].keys())
    body = "\n".join("\t".join(str(v) for v in r) for r in rows)
    return f"{header}\n{body}"

# ======================
# STREAMLIT APP
# ======================
st.set_page_config(page_title="Chinook NL-to-SQL + Meta", layout="wide")
st.title("🎵 Chinook NL-to-SQL + Schema Q&A")
st.caption("Ask about data OR database structure (tables, columns, etc.)")

st.info("""
**Try these:**
- *Data*: "Top 3 customers by total spending"
- *Meta*: "What columns are in the Invoice table?"
- *Meta*: "List all tables in the database"
""")

question = st.text_input("Ask anything about the Chinook database:", 
                        placeholder="e.g., How many tracks are there? OR What is the schema?")

if st.button("Get Answer") and question:
    if not API_KEY or API_KEY == "your-openrouter-api-key":
        st.error("🔑 Set OpenRouter API key in environment variable OPENROUTER_API_KEY")
        st.stop()

    try:
        # === STEP 1: GUARDRAIL ===
        with st.spinner("Validating..."):
            guard_out = call_openrouter(GUARD_MODEL, GUARD_PROMPT.format(question=question))
            if "VALID" not in guard_out.upper():
                st.error("❌ Rejected: off-topic or unsafe.")
                st.stop()

        # === STEP 2: META vs DATA ===
        with st.spinner("Classifying..."):
            meta_out = call_openrouter(META_MODEL, META_PROMPT.format(question=question))
            is_meta = "META" in meta_out.upper()

        if is_meta:
            # === META PATH ===
            with st.spinner("Answering from schema..."):
                natural_answer = call_openrouter(
                    ANSWER_MODEL,
                    ANSWER_FROM_META_PROMPT.format(question=question, schema=SCHEMA_DDL)
                )
                sql_query = None
                results = None
        else:
            # === SQL PATH ===
            with st.spinner("Generating SQL..."):
                sql_raw = call_openrouter(SQL_MODEL, SQL_PROMPT.format(schema=SCHEMA_DDL, question=question))
                sql_match = re.search(r"(SELECT.*?;)", sql_raw, re.DOTALL | re.IGNORECASE)
                sql_query = sql_match.group(1) if sql_match else sql_raw.split("\n")[0]

            with st.spinner("Executing..."):
                results = safe_execute_sql(sql_query)
                tsv_result = format_tsv(results)

            with st.spinner("Generating answer..."):
                natural_answer = call_openrouter(
                    ANSWER_MODEL,
                    ANSWER_FROM_DATA_PROMPT.format(question=question, result=tsv_result)
                )

        # === OUTPUT ===
        st.subheader("✅ Answer")
        st.write(natural_answer)

        if is_meta:
            st.subheader("ℹ️ Answer Source")
            st.write("Schema introspection (no SQL executed)")
        else:
            st.subheader("🔍 Generated SQL")
            st.code(sql_query, language="sql")
            st.subheader("📊 Results")
            if results:
                st.dataframe([dict(r) for r in results], use_container_width=True)
            else:
                st.write("(no results)")

    except Exception as e:
        st.error(f"💥 Error: {e}")

st.markdown("---")
st.caption("Supports both data queries and schema/meta questions • Read-only • Chinook DB")

### Run the Streamlit App

**Note**: Set the `OPENROUTER_API_KEY` environment variable before running.

In [ ]:
# Run the Streamlit app
# First, set your API key in PowerShell:
# $env:OPENROUTER_API_KEY='your-api-key-here'

! streamlit run app.py